In [8]:
numbers = [1,5,8,9,0,3,2,4,6,7]


In [9]:
def classical_oracle(input):
    winner = 4
    if input is winner:
        response = True
    else:
        response = False
    return response

In [10]:
for index, trial_nr in enumerate(numbers):
    if classical_oracle(trial_nr) is True:
        print('Winner found at index %i'%index)
        print('%i calls to the classical oracle'%(index+1))
        break

Winner found at index 7
8 calls to the classical oracle


In [11]:
from pyquil import get_qc, Program
from pyquil.gates import CZ
from pyquil.latex import display
import matplotlib.pyplot as pyplot
import numpy

In [13]:
quantum_oracle = Program()
quantum_oracle += CZ(0,1)
display(quantum_oracle)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 482: invalid start byte